In [66]:
import os
import sys
import numpy as np
import sklearn as skl
import time
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
from sklearn.model_selection import train_test_split

sys.path.append("../../../")
import CalibratedTimeseriesModels

from CalibratedTimeseriesModels.utils import *
from CalibratedTimeseriesModels.models.gnn import *
from CalibratedTimeseriesModels.models.gmnn import *
from CalibratedTimeseriesModels.models.blr import *
from CalibratedTimeseriesModels.evaluators import ExplicitEvaluator

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# load data
X_orig = torch.load("../../../datasets/processed/openEI/X_openei_011_subset_multitask.pt")
Y_orig = torch.load("../../../datasets/processed/openEI/Y_openei_011_subset_multitask.pt")

past_dims = 24
fut_dims = 12

X_train_orig, X_test_orig, Y_train_orig, Y_test_orig = electric_train_test_split(X_orig, Y_orig, 
                                                                                 disp_idx=past_dims+fut_dims)

In [37]:
X_train = X_train_orig[1,:,:past_dims].reshape((-1,past_dims)).unsqueeze(-1).float()
Y_train_full = Y_train_orig[1,:,:fut_dims].reshape((-1,fut_dims)).unsqueeze(-1).float()
X_test = X_test_orig[1,:,:past_dims].reshape((-1,past_dims)).unsqueeze(-1).float()
Y_test_full = Y_test_orig[1,:,:fut_dims].reshape((-1,fut_dims)).unsqueeze(-1).float()

Y_train = Y_train_full[:,[0],:]
Y_test = Y_test_full[:,[0],:]

X_batches, Y_batches = batch(X_train, Y_train, batch_size = 128)

### Single Step LR

In [38]:
lin_reg = BayesianLinearRegression(1, past_dims, 1, 1)
lin_reg.fit(X_train, Y_train)
dtest0 = lin_reg(X_test)
for f in [mape, wape, rmse, rwse, nll]:
    print(f(dtest0,Y_test))

(tensor(0.0498), tensor([0.0498]), tensor([0.0485]))
(tensor(0.0755), tensor([0.0755]), tensor([0.0436]))
(tensor(0.0740), tensor([0.0055]), tensor([0.0109]))
(tensor(0.0967), tensor([0.0094]), tensor([0.0108]))
(tensor(-1.1512), tensor([-1.8127, -1.8539, -1.6309,  ..., -1.3570, -0.6578, -1.0662]))


### Single Step GNN

In [41]:
hidden_layers = [40, 40, 40]
ss_gnn = GaussianNeuralNet(1, past_dims, hidden_layers, 1, 1)
train(ss_gnn, X_batches, Y_batches, num_epochs=150, learning_rate=.005)
dtest_gnn = ss_gnn(X_test)
for f in [mape, wape, rmse, rwse, nll]:
    print(f(dtest_gnn,Y_test))

epoch : 1, loss: 0.4372
epoch : 11, loss: -1.3281
epoch : 21, loss: -1.3911
epoch : 31, loss: -1.4123
epoch : 41, loss: -1.5076
epoch : 51, loss: -1.4680
epoch : 61, loss: -1.7906
epoch : 71, loss: -1.6621
epoch : 81, loss: -1.7677
epoch : 91, loss: -1.9560
epoch : 101, loss: -1.8870
epoch : 111, loss: -1.7548
epoch : 121, loss: -2.0011
epoch : 131, loss: -1.9669
epoch : 141, loss: -1.8842
Learning finished!
(tensor(0.0338, grad_fn=<MeanBackward0>), tensor([0.0338], grad_fn=<MeanBackward1>), tensor([0.0385], grad_fn=<StdBackward1>))
(tensor(0.0452), tensor([0.0452]), tensor([0.0348]))
(tensor(0.0573, grad_fn=<PowBackward0>), tensor([0.0033], grad_fn=<MeanBackward1>), tensor([0.0082], grad_fn=<StdBackward1>))
(tensor(0.0754), tensor([0.0057]), tensor([0.0096]))
(tensor(-1.6022, grad_fn=<MeanBackward0>), tensor([-3.1669, -3.1733, -2.9779,  ..., -0.1438, -0.7907, -1.7754],
       grad_fn=<SumBackward1>))


### Single Step GMNN

In [42]:
hidden_layers = [40, 40, 40]
ss_gmnn = GaussianMixtureNeuralNet(1, past_dims, hidden_layers, 1, 1, n_components=3)
train(ss_gmnn, X_batches, Y_batches, num_epochs=150, learning_rate=.005)
dtest_gmnn = ss_gmnn(X_test)
for f in [mape, wape, rmse, rwse, nll]:
    print(f(dtest_gmnn,Y_test))

epoch : 1, loss: 0.7016
epoch : 11, loss: -1.2484
epoch : 21, loss: -1.4711
epoch : 31, loss: -1.6136
epoch : 41, loss: -1.6917
epoch : 51, loss: -1.6859
epoch : 61, loss: -1.7611
epoch : 71, loss: -1.9041
epoch : 81, loss: -1.8746
epoch : 91, loss: -1.9587
epoch : 101, loss: -1.8054
epoch : 111, loss: -1.9890
epoch : 121, loss: -1.8087
epoch : 131, loss: -1.6759
epoch : 141, loss: -2.0839
Learning finished!
(tensor(0.0343, grad_fn=<MeanBackward0>), tensor([0.0343], grad_fn=<MeanBackward1>), tensor([0.0377], grad_fn=<StdBackward1>))
(tensor(0.0547), tensor([0.0547]), tensor([0.0334]))
(tensor(0.0575, grad_fn=<PowBackward0>), tensor([0.0033], grad_fn=<MeanBackward1>), tensor([0.0068], grad_fn=<StdBackward1>))
(tensor(0.0877), tensor([0.0077]), tensor([0.0088]))
(tensor(-1.6733, grad_fn=<MeanBackward0>), tensor([-2.7498, -2.8066, -2.7430,  ..., -1.3938, -2.2997, -2.0622],
       grad_fn=<NegBackward>))


### Single Step GLSTM

In [64]:
hidden_layers = [20, 20, 20]
hidden_dim = 40
ss_glstm = GaussianLSTM(1, hidden_dim, hidden_layers, 1, 1)
train(ss_glstm, X_batches, Y_batches, num_epochs=120, learning_rate=.005)
dtest_glstm = ss_glstm(X_test)
for f in [mape, wape, rmse, rwse, nll]:
    print(f(dtest_glstm,Y_test))

epoch : 1, loss: 0.9823
epoch : 11, loss: -0.5173
epoch : 21, loss: -1.0173
epoch : 31, loss: -1.1674
epoch : 41, loss: -1.4493
epoch : 51, loss: -1.5867
epoch : 61, loss: -1.6917
epoch : 71, loss: -1.6981
epoch : 81, loss: -1.8621
epoch : 91, loss: -1.9247
epoch : 101, loss: -2.0530
epoch : 111, loss: -2.0541
Learning finished!
(tensor(0.0354, grad_fn=<MeanBackward0>), tensor([0.0354], grad_fn=<MeanBackward1>), tensor([0.0427], grad_fn=<StdBackward1>))
(tensor(0.0453), tensor([0.0453]), tensor([0.0404]))
(tensor(0.0619, grad_fn=<PowBackward0>), tensor([0.0038], grad_fn=<MeanBackward1>), tensor([0.0098], grad_fn=<StdBackward1>))
(tensor(0.0780), tensor([0.0061]), tensor([0.0123]))
(tensor(-1.3906, grad_fn=<MeanBackward0>), tensor([-3.4392, -3.5476, -3.3663,  ...,  1.6993, -2.8094, -2.4952],
       grad_fn=<NegBackward>))


### Single Step GMM LSTM

In [68]:
hidden_layers = [20, 20, 20]
hidden_dim = 40
ss_gmmlstm = GaussianMixtureLSTM(1, hidden_dim, hidden_layers, 1, 1, n_components=2)
train(ss_gmmlstm, X_batches, Y_batches, num_epochs=120, learning_rate=.005)
dtest_gmmlstm = ss_gmmlstm(X_test)
for f in [mape, wape, rmse, rwse, nll]:
    print(f(dtest_gmmlstm,Y_test))

epoch : 1, loss: 0.9227
epoch : 11, loss: -1.1746
epoch : 21, loss: -1.2586
epoch : 31, loss: -1.3158
epoch : 41, loss: -1.3251
epoch : 51, loss: -1.3531
epoch : 61, loss: -1.4063
epoch : 71, loss: -1.4360
epoch : 81, loss: -1.4693
epoch : 91, loss: -1.5216
epoch : 101, loss: -1.5683
epoch : 111, loss: -1.5885
Learning finished!
(tensor(0.0791, grad_fn=<MeanBackward0>), tensor([0.0791], grad_fn=<MeanBackward1>), tensor([0.0913], grad_fn=<StdBackward1>))
(tensor(0.1035), tensor([0.1035]), tensor([0.0848]))
(tensor(0.1387, grad_fn=<PowBackward0>), tensor([0.0192], grad_fn=<MeanBackward1>), tensor([0.0408], grad_fn=<StdBackward1>))
(tensor(0.1839), tensor([0.0338]), tensor([0.0506]))
(tensor(-0.9125, grad_fn=<MeanBackward0>), tensor([-2.4752, -2.6291, -2.5837,  ..., -1.0152, -1.3710, -1.5033],
       grad_fn=<NegBackward>))


### Forward Sampling:

In [ ]:
def sample_forward_nn(model, y, prediction_horizon, n_samples=1000):
    
    # initial step
    sequence = [model(y).sample((n_samples,))] #(n_samples, B, 1)
    
    # append to front of input sequence
    
    for i in range(1, prediction_horizon):

        # run through model
        dist = model(y)
        
        # generate next time series
        
        # append latest samples to front of sequence (OPENEI DATA)
        y = torch.cat((),)
        
    samples = torch.cat(sequence,2)
    return samples


In [ ]:
def sample_forward_lstm(model, y, n_samples=1000):
    pass